# The Adjunction of the Fold
###### (https://blog.jle.im/entry/foldl-adjunction.html)
The `foldl` library allow for the respresentation of folds as a value.

In [15]:
:ext ExistentialQuantification

data Fold a r = forall x. Fold (x -> a -> x) x (x -> r)

instance Functor (Fold a) where
  fmap f (Fold c x xr) = Fold c x (f . xr)
  
instance Applicative (Fold a) where
  pure r = Fold (\x a -> x ++ [a]) [] (const r)
  (Fold c1 x1 f1) <*> (Fold c2 x2 f2) = Fold (\(x1, x2) a -> (c1 x1 a, c2 x2 a)) (x1, x2) (\(x1, x2) -> f1 x1 $ f2 x2)
  
fold :: Fold a b -> [a] -> b
fold (Fold step begin end) as = foldr go end as begin where
  go a k x = k $ step x a

The `Fold` Functor is representable so we expect it to the right adjoint to some other Functor.

In [16]:
:ext DeriveFunctor

data EnvList a r = EnvList [a] r deriving Functor

instance Applicative (EnvList a) where
  pure = EnvList []
  (EnvList as1 f) <*> (EnvList as2 x) = EnvList (as1 ++ as2) (f x)

Show that this forms an adjunction

In [17]:
:ext MultiParamTypeClasses

-- g is also Representable
class (Functor f, Functor g) => Adjunction f g where
  leftAdjunct :: (f a -> b) -> (a -> g b)
  leftAdjunct fab a = fab <$> unit a
  rightAdjunct :: (a -> g b) -> (f a -> b)
  rightAdjunct agb fa = counit $ agb <$> fa
  unit :: a -> g (f a)
  unit = leftAdjunct id
  counit :: f (g a) -> a
  counit = rightAdjunct id
  
instance Adjunction (EnvList a) (Fold a) where
  leftAdjunct envListToB a = Fold step [] (envListToB . (`EnvList` a)) where
    step xs a = xs ++ [a]
  rightAdjunct aToFoldB (EnvList xs a) = fold (aToFoldB a) xs

Show that the right Kan extension of the Identity Functor over Fold is isomorphic to `EnvList`

In [18]:
newtype RightAdj f a = RightAdj { runRightAdj :: forall b. (a -> f b) -> b } deriving Functor

type FoldAdj a r = RightAdj (Fold a) r

toEnvList :: FoldAdj a r -> EnvList a r
toEnvList (RightAdj rToFoldBToB) = rToFoldBToB $ leftAdjunct id

fromEnvList :: EnvList a r -> FoldAdj a r
fromEnvList envList = RightAdj $ \rToFoldB -> rightAdjunct rToFoldB envList

Show that a Fold of an EnvList is a Monad

In [19]:
:ext FlexibleInstances
:m Control.Monad

newtype Compose f g a = Compose { unCompose :: f (g a) } deriving Functor

instance (Applicative f, Applicative g) => Applicative (Compose f g) where
  pure = Compose . pure . pure
  (Compose fgf) <*> (Compose fga) = Compose $ (<*>) <$> fgf <*> fga

instance Monad (Compose (Fold a) (EnvList a)) where
  return = pure
  c >>= f = Compose . fmap counit . unCompose . fmap unCompose $ f <$> c

Show that EnvList of a Fold is a Comonad

In [23]:
class Functor w => Comonad w where
  extract :: w a -> a
  extend :: (w a -> b) -> w a -> w b
  extend f = fmap f . duplicate
  duplicate :: w a -> w (w a)
  duplicate = extend id
  
instance Comonad (Compose (EnvList a) (Fold a)) where
  extract (Compose (EnvList as f)) = fold f as
  duplicate = fmap Compose . Compose . fmap unit . unCompose